In [1]:
!pip install -q langchain langchain-community chromadb sentence-transformers openai tiktoken python-dotenv langchain_openai

In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from typing import Dict, Any

In [4]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY")
project_id = os.getenv("PROJECT_ID")

client = OpenAI(api_key=api_key, project=project_id)

In [8]:
%pip install -qU pypdf

Note: you may need to restart the kernel to use updated packages.


In [46]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader


file_path = "data/"
loader = PyPDFDirectoryLoader(file_path)
docs = loader.load()
print(len(docs))
docs[144]
# loader = PyPDFLoader(file_path)
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

291


Document(metadata={'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'In-Class Review: The Foundations of Linear Regression', 'source': 'data/In-Class Review_ The Foundations of Linear Regression - P2W1D4 - 6:12.pdf', 'total_pages': 4, 'page': 1, 'page_label': '2'}, page_content='●  Predicting  housing  prices  based  on  square  footage  and  number  of  bedrooms  \n ●  Predicting  health  outcomes  based  on  age,  BMI,  and  blood  pressure  \n \nDiscussion  Prompt:  \n \nHow\n \nis\n \nthis\n \ndifferent\n \nfrom\n \nwriting\n \nan\n \nexplicit\n \nformula\n \nor\n \nalgorithm?\n \n \n2 ⃣  Optimization  Warm-Up:  The  Minecraft  Backyard  Problem  \nProblem:  \n \nYou\n \nhave\n \n20\n \nfence\n \nblocks\n \nto\n \nbuild\n \na\n \nrectangular\n \n"Minecraft"\n \nbackyard.\n \n \nThe\n \nconstraint\n \nis:\n \n2L+W=20  \nYou  want  to  maximize  the  area :  \nA=L×W  \nSubstitute  W  =  20  -  2L:  \nA  =  L(20  -  2L)      =  20L  - 

In [47]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=api_key)

solutions_db = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="./solutions_index")
solutions_db.persist()

In [48]:
solutions_retriever = solutions_db.as_retriever(search_kwargs={"k": 3})

In [65]:
def solution_proposer(student_bug: str) -> str:

    context = solutions_retriever.get_relevant_documents(student_bug)
    prompt = f"""
        Student Error Message: {student_bug} 
        You are a master expert at teaching, implementing code and debugging code for students at a data science fellowship.
        Use the solutions context which contains class slides and PDF to point students to where they can refer to
        to find the solution or topic to their question. If possible include phase, week and day and page number - extract this information from the title of the pdf and display it like Phase 1, Week 1, Day 1 (P represents Phase, W represents week and D represents day)
        {context}
        """

    resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
        )
    return resp.choices[0].message.content

In [64]:
def solution_checker(student_bug: str, solution: str) -> str: 
    prompt = f"""
        Review this response and make sure the answer was not directly given to the student.
        If it does, please update it and remove any given solutions.
        You should point to resources and only guide student and give hints to the solution. 
        This is the student's error message: {student_bug}
        This is the response {solution}
        Direct this message to the student.
    """

    resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
        )

    return resp.choices[0].message.content

In [ ]:
student_issue = f"""
    adult_trips = dfclean[df_clean["___" > 2
    adult_trips
    '[' was never closed
"""
# student_bug = input("Please enter your bug")

In [66]:
def handle_bug(issue: str):
    print("Student bug:", issue, "\n")
    draft = solution_proposer(student_issue)
    print("Proposed Solution:", draft, "\n")
    final_response = solution_checker(student_issue, draft)
    print("Debugger:", final_response, "\n")

In [67]:
handle_bug(student_issue)

Student bug: 
    adult_trips = dfclean[df_clean["___" > 2
    adult_trips
    '[' was never closed
 

Proposed Solution: It looks like you're encountering a syntax error in your code. The error message indicates that there is an issue with the way you've structured your conditional statement. Specifically, it seems that you have a typo in your code where you are trying to filter the DataFrame.

Here’s the corrected version of your code:

```python
adult_trips = df_clean[df_clean["column_name"] > 2]
```

Make sure to replace `"column_name"` with the actual name of the column you want to filter on. Also, ensure that you have closed all brackets properly.

For more information on how to filter DataFrames in Pandas, you can refer to the class slides. Specifically, you can find relevant content on row indexing and filtering in the following document:

**Phase 2, Week 1, Day 1 (P2W1D1)** - Page 77 of the "Introduction to Phase 2 & Python Refresher" presentation discusses row indexing and fi